In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./7.page.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Type No'] = input_['Type No'].astype(int)
input_['SKU Count'] = input_['SKU Count'].astype(int)
input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            list_url = []
            while len(list_url) < input_.loc[a, 'SKU Count']:
                for i in range(input_.loc[a, 'Page']):
                    b = 0
                    while True:
                        b += 1

                        try:
                            resp = requests.get(f'''{input_.loc[a, 'Type Url']}&start={i*100}&num=100''',
                                                headers=get_header(),
                                                proxies=get_proxy(),
                                                timeout=(10, 10))

                            if resp.status_code == 200:
                                break
                        except:
                            continue
                
                            if resp.status_code == 200:
                                break
            
                    # = = = = = = = = = = = = = = =
                        
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))
                
                    # = = = = = = = = = = = = = = =
                
                    list_url += [f'https://www.dormanproducts.com/{href.strip()}' for href in html.xpath('//h2[@class="item-headline"]/a/@href')]
                    list_url = sorted(list(set(list_url)), key=list_url.index)
    
                    # = = = = = = = = = = = = = = =

                    if work.qsize() % 100 == 0 and work.qsize() != 0:
                        clear_output()
                    
                    print(f'''[尝试次数：{b}] - {input_.loc[a, 'Type No']}.{input_.loc[a, 'Type']} > Page {i+1}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

                    # = = = = = = = = = = = = = = =

                    if len(list_url) == input_.loc[a, 'SKU Count']:
                        break

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Type No': input_.loc[a, 'Type No'],
                                    'Type': input_.loc[a, 'Type'],
                                    'Type Url': input_.loc[a, 'Type Url'],
                                    'SKU Count': input_.loc[a, 'SKU Count'],
                                    'Page': input_.loc[a, 'Page'],
                                    'No': [i+1 for i in range(len(list_url))],
                                    'Url': list_url})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Type No': input_.loc[a, 'Type No'],
                                     'Type': input_.loc[a, 'Type'],
                                     'Type Url': input_.loc[a, 'Type Url'],
                                     'SKU Count': input_.loc[a, 'SKU Count'],
                                     'Page': input_.loc[a, 'Page']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Type No', 'No'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./8.url_1-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Type No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./url_1_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[尝试次数：50] - 2173.Vacuum Reservoir Nut > Page 1
[剩余数量：100] - [当前时间：16:01:20]

[尝试次数：1] - 1849.Suspension - Control Arm Bushing > Page 78
[剩余数量：99] - [当前时间：16:01:21]

[尝试次数：1] - 1855.Suspension - Control Arm and Ball Joint Assembly > Page 84
[剩余数量：99] - [当前时间：16:01:21]

[尝试次数：1] - 1842.Suspension - Control Arm > Page 80
[剩余数量：99] - [当前时间：16:01:22]

[尝试次数：65] - 1844.Suspension - Control Arm Ball Bushing > Page 71
[剩余数量：99] - [当前时间：16:01:23]

[尝试次数：1] - 2163.Vacuum Check Valve > Page 3
[剩余数量：99] - [当前时间：16:01:23]

[尝试次数：2] - 2165.Vacuum Hose > Page 2
[剩余数量：99] - [当前时间：16:01:23]

[尝试次数：1] - 1983.Tailgate Latch Actuator > Page 5
[剩余数量：99] - [当前时间：16:01:24]

[尝试次数：2] - 2103.Turbocharger > Page 11
[剩余数量：99] - [当前时间：16:01:25]

[尝试次数：1] - 1293.Mirror Assembly - Door > Page 17
[剩余数量：99] - [当前时间：16:01:25]

[尝试次数：1] - 1982.Tailgate Latch > Page 2
[剩余数量：99] - [当前时间：16:01:25]

[尝试次数：41] - 1781.Steering - Tie Rod End Assembly > Page 30
[剩余数量：99] - [当前时间：16:01:27]

[尝试次数：35] - 1778.Steering - Tie Rod E